假设我是一名证券分析师，我跟踪了很多支股票，每一支股价信息存在不同的csv表格里面。
现在我要，把不同股票的信息(AAPL.csv, AMAZ.csv等等，文件名称是股票代码)，合并到一个excel文件里（如stock_price.xlsx）。
在合并文件里，每一个sheet都是不同的股票信息，sheet的名字是股票代码。
在一个sheet中，有7列，分别是交易日期，开盘价，当日最高价，最低价，复权后的最高，最低价，以及成交量。

需要做的事情有：
1. 利用其他的excel表格，生成stock_price.xlsx这样的一个合并表格
2. 算出每一支股票在每一个自然月中价格变动幅度（百分数），另存成一个sheet
3. 算出每支股票近一年的涨幅（百分数），生成一个新sheet，两列，第一列是股票代码，第二列是涨幅，按照涨幅从大到小排序

合并出来的表可能很大，excel加载慢，复杂运算可能造成司机，python读取和操作更快
这个可能也可以用在银行客户管理上，可以用于合并每一个客户的存取款信息及账户余额信息，计算每月平均存款数值，检测账户金额异常变动等

In [241]:
# 头文件的引用
import pandas as pd
import os

## 个股csv文件读取

In [242]:
data = dict()
for file in os.listdir('./'):
    file_split = file.split('.')
    if file_split[-1]=='csv':
        f = pd.read_csv(file_split[0]+".csv")
        data[file_split[0]] = f

In [243]:
print(type(data))
stock_names = data.keys()
stock_names

<class 'dict'>


dict_keys(['AMZN', 'MSFT', 'FB', 'NFLX', 'TSLA', 'GOOG', 'BABA', 'AAPL'])

In [244]:
type(data['AMZN'])

pandas.core.frame.DataFrame

## 文件数据概述

In [245]:
example=data['AMZN'].copy()

In [246]:
example.head(2)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014/11/28,539.139771,540.515991,535.130798,540.346497,540.346497,1148300
1,2014/12/1,537.424500,539.927612,530.403748,532.338440,532.338440,2115300


In [247]:
example.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [248]:
example.describe()

,Open,High,Low,Close,Adj Close,Volume
count,1259.000000,1259.000000,1259.000000,1259.000000,1259.000000,1.259000e+03
mean,898.777303,906.494730,891.034000,899.057177,899.057177,1.720754e+06
std,229.229864,231.481069,227.464112,229.603040,229.603040,8.517839e+05
min,493.295654,494.618011,486.225067,491.201416,491.201416,5.272000e+05
25%,722.109985,726.404999,715.814972,719.970001,719.970001,1.217600e+06
50%,922.530029,926.549988,915.500000,921.809998,921.809998,1.496800e+06
75%,1104.205017,1112.838012,1094.605041,1103.614990,1103.614990,1.968500e+06
max,1332.219971,1335.529053,1317.500000,1334.869995,1334.869995,1.116490e+07


In [249]:
# 上面没有Date数据描述的原因是因为Date不是数字类型，是string object
example.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

#### 我们会想把日期转换为Python专门的Date格式，因为Python集成了许多对日期的操作可以直接使用，比如说日期可以直接比较大小，可以固定间隔采样日期等操作，我们就不需要自己考虑年月日之间的关系了。

In [250]:
# 此时Date列里是字符串
example['Date'][0]

'2014/11/28'

In [251]:
example['Date'] = pd.to_datetime(example['Date'], format="%Y/%m/%d")

In [252]:
# 这时Date里已经是Python的日期格式了
example.dtypes

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object

In [253]:
# 将日期设为表格的y轴(index轴)
example = example.set_index('Date')
example.head(2)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-11-28,539.139771,540.515991,535.130798,540.346497,540.346497,1148300
2014-12-01,537.424500,539.927612,530.403748,532.338440,532.338440,2115300


In [254]:
# 日期可以方便的直接比较
example.loc[example.index < '2014-12-03']

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-11-28,539.139771,540.515991,535.130798,540.346497,540.346497,1148300
2014-12-01,537.424500,539.927612,530.403748,532.338440,532.338440,2115300
2014-12-02,532.049255,534.033813,528.349426,532.288574,532.288574,1526600


In [255]:
# 现在我们把上面的代码整理一下，写成一个对每只股票数据整理的函数
def readData(stock_data: pd.DataFrame) -> pd.DataFrame:
    """
    Read the date column into datetime64, and make this column the index
    Args:
        stock_data: A dataframe that contains ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    Returns:
        Returns a dataframe with the date as row index
    """    
    stock_data['Date'] = pd.to_datetime(stock_data['Date'], format="%Y/%m/%d")
    stock_data = stock_data.set_index('Date')
    return stock_data

In [256]:
# 我们把每一个股票的数据都处理一下
for stock_code in stock_names:
    data[stock_code] = readData(data[stock_code])

#### 让我们看看Python日期模块有多强大，可以简单完成任意时间内的价格变动分析

In [257]:
# 看每两周的数据
example.resample('2W').first().head(2)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-11-30,539.139771,540.515991,535.130798,540.346497,540.346497,1148300
2014-12-14,537.424500,539.927612,530.403748,532.338440,532.338440,2115300


In [258]:
# 以收盘价为参考看每个月价格的变动幅度
example['Close'].resample('1M').first()

Date
2014-11-30     540.346497
2014-12-31     532.338440
2015-01-31     523.373108
2015-02-28     527.033020
2015-03-31     569.775696
                 ...     
2019-07-31    1097.949951
2019-08-31    1209.010010
2019-09-30    1168.390015
2019-10-31    1205.099976
2019-11-30    1273.739990
Freq: M, Name: Close, Length: 61, dtype: float64

In [259]:
# 甚至是看最近的52周（一年）之内每个月的价格变动幅度
example['Close'].last('52W').resample('1M').first()

Date
2018-12-31    1106.430054
2019-01-31    1045.849976
2019-02-28    1110.750000
2019-03-31    1140.989990
2019-04-30    1194.430054
2019-05-31    1168.079956
2019-06-30    1036.229980
2019-07-31    1097.949951
2019-08-31    1209.010010
2019-09-30    1168.390015
2019-10-31    1205.099976
2019-11-30    1273.739990
Freq: M, Name: Close, dtype: float64

In [260]:
# 有现成的函数ohlc()代表每个间隔内的open, high, low, close信息
monthly_ohlc = example['Close'].resample('1M').ohlc()
monthly_ohlc

,open,high,low,close
Date,,,,
2014-11-30,540.346497,540.346497,540.346497,540.346497
2014-12-31,532.338440,535.838867,494.033630,524.958740
2015-01-31,523.373108,538.471619,491.201416,533.056519
2015-02-28,527.033020,556.871094,521.328674,556.871094
2015-03-31,569.775696,573.754761,545.821472,546.499573
...,...,...,...,...
2019-07-31,1097.949951,1250.410034,1097.949951,1216.680054
2019-08-31,1209.010010,1209.010010,1151.290039,1188.099976
2019-09-30,1168.390015,1246.520020,1168.390015,1219.000000


这个表格每行的意思是，比如在2014年12月（11月只有一天所以用12月做例子）里收盘价从532跌倒了524，这个月里最高达到了525最低494，是不是非常简单。

这里如果我们想在表格里加一列表示波动幅度百分比呢？

In [261]:
monthly_ohlc['fluctuation %'] = (monthly_ohlc['close']-monthly_ohlc['open'])/monthly_ohlc['open']*100

In [262]:
monthly_ohlc.head(4)

,open,high,low,close,fluctuation %
Date,,,,,
2014-11-30,540.346497,540.346497,540.346497,540.346497,0.000000
2014-12-31,532.338440,535.838867,494.033630,524.958740,-1.386280
2015-01-31,523.373108,538.471619,491.201416,533.056519,1.850193
2015-02-28,527.033020,556.871094,521.328674,556.871094,5.661519


In [263]:
# 整理上面的代码，写一个函数可以生成每个股票的monthly浮动
def get_monthly(stock_data: pd.DataFrame) -> pd.DataFrame:
    monthly = stock_data['Close'].resample('1M').ohlc()
    monthly['fluctuation %'] = (monthly['close']-monthly['open'])/monthly['open']*100
    return monthly

In [264]:
for stock_code in list(stock_names):
    data[stock_code+"_monthly"] = get_monthly(data[stock_code])
data.keys()

dict_keys(['AMZN', 'MSFT', 'FB', 'NFLX', 'TSLA', 'GOOG', 'BABA', 'AAPL', 'AMZN_monthly', 'MSFT_monthly', 'FB_monthly', 'NFLX_monthly', 'TSLA_monthly', 'GOOG_monthly', 'BABA_monthly', 'AAPL_monthly'])

## 获取这一自然年内的股价

In [154]:
# 获取当前年份
year_now = example.last('1D').index[0].year
year_now

2019

In [155]:
a_year_data = example[example.index.year == year_now]
a_year_data.head(5)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,1016.570007,1052.319946,1015.710022,1045.849976,1045.849976,1532600
2019-01-03,1041.000000,1056.979980,1014.070007,1016.059998,1016.059998,1841100
2019-01-04,1032.589966,1070.839966,1027.417969,1070.709961,1070.709961,2093900
2019-01-07,1071.500000,1074.000000,1054.760010,1068.390015,1068.390015,1981900
2019-01-08,1076.109985,1084.560059,1060.530029,1076.280029,1076.280029,1764900


In [159]:
yearly_ohlc = a_year_data['Close'].resample('1Y').ohlc()
yearly_ohlc

,open,high,low,close
Date,,,,
2019-12-31,1045.849976,1334.869995,1016.059998,1312.98999


In [230]:
# 加上浮动百分比，公司名称，与年份
yearly_ohlc['fluctuation %'] = (yearly_ohlc['close']-yearly_ohlc['open'])/yearly_ohlc['open']*100
yearly_ohlc["company_code"] = 'AMZN'
yearly_ohlc["year"] = year_now
yearly_ohlc.reset_index(drop=True,inplace=True)
yearly_ohlc

,open,high,low,close,fluctuation %,company_code,year
0,1045.849976,1334.869995,1016.059998,1312.98999,25.542862,AMZN,2019


In [229]:
# 这时我们编写一个函数将将所有公司的这个信息拼起来，就是上面的代码和起来
def thisYearChanges(data):
    all_stocks = pd.DataFrame()
    for stock_code in stock_names:
        stock_data = data[stock_code]
        # 获取当前年份
        year_now = stock_data.last('1D').index[0].year
        # 只选用今年的数据
        a_year_data = stock_data[stock_data.index.year == year_now]
        # 今年一整年收盘价的汇总
        yearly_ohlc = a_year_data['Close'].resample('1Y').ohlc()
        # 添加浮动百分比，公司名称与年份到表格中
        yearly_ohlc['fluctuation%'] = (yearly_ohlc['close']-yearly_ohlc['open'])/yearly_ohlc['open']*100
        yearly_ohlc["company_code"] = stock_code
        yearly_ohlc["year"] = year_now
        # 去掉Date信息因为不再重要了
        yearly_ohlc.reset_index(drop=True,inplace=True)
        
        all_stocks = all_stocks.append(yearly_ohlc)
    return all_stocks
trend = thisYearChanges(data).sort_values(by=["fluctuation%"], ascending=False)

In [228]:
trend

,open,high,low,close,fluctuation%,company_code,year
0,310.119995,359.519989,178.970001,328.920013,-6.062175,TSLA,2019
0,267.660004,385.029999,254.589996,315.929993,-18.034069,NFLX,2019
0,1045.849976,1334.869995,1016.059998,1312.989990,-25.542862,AMZN,2019
0,1045.849976,1334.869995,1016.059998,1312.989990,-25.542862,GOOG,2019
0,136.699997,200.820007,130.600006,200.820007,-46.905641,BABA,2019
0,135.679993,204.869995,131.740005,202.000000,-48.879725,FB,2019
0,101.120003,152.320007,97.400002,152.320007,-50.632914,MSFT,2019
0,157.919998,267.839996,142.190002,267.839996,-69.604863,AAPL,2019


## 下面我们来把我们处理好的data变量和trend变量储存到一个excel的不同tab里面去

In [267]:
output_file_name = "cleaned_data.xlsx"

In [269]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(output_file_name, engine='xlsxwriter')

# 写入data
for stock_code in data.keys():
    data[stock_code].to_excel(writer, sheet_name=stock_code)

#写入trend
trend.to_excel(writer, sheet_name="Trend", index=False)

writer.save()

## 代码纯享

下面我们把这整个教程之中的有效代码提取出来，供大家快速翻阅（只是从上面复制粘贴下来的）

In [287]:
# 头文件的引用
import pandas as pd
import os

In [288]:
# 读取各个股票csv文件
data = dict()
for file in os.listdir('./'):
    file_split = file.split('.')
    if file_split[-1]=='csv':
        f = pd.read_csv(file_split[0]+".csv")
        data[file_split[0]] = f
stock_names = data.keys()

In [289]:
# 我们将每个日期字符串转换成Python的日期格式
def readData(stock_data: pd.DataFrame) -> pd.DataFrame:
    # Read the date column into datetime64, and make this column the index
    stock_data['Date'] = pd.to_datetime(stock_data['Date'], format="%Y/%m/%d")
    stock_data = stock_data.set_index('Date')
    return stock_data

# 我们把每一个股票的数据都处理一下
for stock_code in stock_names:
    data[stock_code] = readData(data[stock_code])

In [290]:
# 分析近一个自然年之内所有股票的波动幅度
def thisYearChanges(data):
    all_stocks = pd.DataFrame()
    for stock_code in stock_names:
        stock_data = data[stock_code]
        # 获取当前年份
        year_now = stock_data.last('1D').index[0].year
        # 只选用今年的数据
        a_year_data = stock_data[stock_data.index.year == year_now]
        # 今年一整年收盘价的汇总
        yearly_ohlc = a_year_data['Close'].resample('1Y').ohlc()
        # 添加浮动百分比，公司名称与年份到表格中
        yearly_ohlc['fluctuation%'] = (yearly_ohlc['close']-yearly_ohlc['open'])/yearly_ohlc['open']*100
        yearly_ohlc["company_code"] = stock_code
        yearly_ohlc["year"] = year_now
        # 去掉Date信息因为不再重要了
        yearly_ohlc.reset_index(drop=True,inplace=True)
        
        all_stocks = all_stocks.append(yearly_ohlc)
    return all_stocks
trend = thisYearChanges(data).sort_values(by=["fluctuation%"], ascending=False)

In [291]:
# 写一个函数可以生成每个股票的monthly浮动
def get_monthly(stock_data: pd.DataFrame) -> pd.DataFrame:
    monthly = stock_data['Close'].resample('1M').ohlc()
    monthly['fluctuation %'] = (monthly['close']-monthly['open'])/monthly['open']*100
    return monthly
for stock_code in list(stock_names):
    data[stock_code+"_monthly"] = get_monthly(data[stock_code])

In [292]:
# 保存至excel中
output_file_name = "cleaned_data.xlsx"
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(output_file_name, engine='xlsxwriter')

# 写入data
for stock_code in data.keys():
    data[stock_code].to_excel(writer, sheet_name=stock_code)

#写入trend
trend.to_excel(writer, sheet_name="Trend", index=False)

writer.save()